In [26]:
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox, Scrollbar
from PIL import Image, ImageTk, ImageDraw, ImageFont
from fpdf import FPDF
import pydicom
import os
import pickle
import numpy as np
from ultralytics import YOLO

class MedImagePro:
    def __init__(self, root):
        self.root = root
        self.root.title("MedImagePro")

        # Load YOLOv8 model
        self.model = YOLO('yolov8l.pt')

        # Operation box at the top
        self.operation_frame = tk.Frame(root)
        self.operation_frame.pack(side=tk.TOP, fill=tk.X)

        self.load_button = tk.Button(self.operation_frame, text="Load Image", command=self.load_image)
        self.load_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.process_button = tk.Button(self.operation_frame, text="Apply Filter", command=self.apply_filter)
        self.process_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.undo_button = tk.Button(self.operation_frame, text="Undo", command=self.undo)
        self.undo_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.restore_button = tk.Button(self.operation_frame, text="Restore", command=self.restore)
        self.restore_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.export_button = tk.Button(self.operation_frame, text="Export as PDF", command=self.export_as_pdf)
        self.export_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.export_dicom_button = tk.Button(self.operation_frame, text="Export as DICOM", command=self.export_modified_dicom)
        self.export_dicom_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.save_button = tk.Button(self.operation_frame, text="Save Project", command=self.save_project)
        self.save_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.load_project_button = tk.Button(self.operation_frame, text="Load Project", command=self.load_project)
        self.load_project_button.pack(side=tk.LEFT, padx=5, pady=5)

        # Main frame for images and zoom controls
        self.main_frame = tk.Frame(root)
        self.main_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # Toggle button for DICOM details
        self.details_toggle_button = tk.Button(self.main_frame, text="⮞", command=self.toggle_details_visibility)
        self.details_toggle_button.pack(side=tk.LEFT, padx=5, pady=5)

        # Details Frame on the left, initially hidden
        self.details_frame = tk.Frame(self.main_frame, width=200)
        self.details_frame.pack(side=tk.LEFT, fill=tk.Y)
        self.details_frame.pack_forget()  # Initially hidden

        self.details_label = tk.Label(self.details_frame, text="DICOM Details")
        self.details_label.pack(side=tk.TOP, anchor="w", padx=5, pady=5)

        self.details_text = tk.Text(self.details_frame, height=15, width=30)
        self.details_text.pack(side=tk.TOP, fill=tk.Y, padx=5, pady=5, expand=True)

        self.details_frame_visible = False

        # Images Frame
        self.images_frame = tk.Frame(self.main_frame)
        self.images_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Original Image Frame
        self.original_frame = tk.Frame(self.images_frame, width=100, height=100, bg="white")
        self.original_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.zoom_scale = tk.Scale(self.images_frame, from_=1, to=5, orient=tk.VERTICAL, resolution=0.1, command=self.zoom_images)
        self.zoom_scale.pack(side=tk.LEFT, fill=tk.X, padx=5, pady=5)
        self.zoom_scale.set(1)

        self.original_canvas = tk.Canvas(self.original_frame, bg="grey")
        self.original_canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # Processed Image Frame
        self.processed_frame = tk.Frame(self.images_frame, width=100, height=100, bg="white")
        self.processed_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.processed_canvas = tk.Canvas(self.processed_frame, bg="grey", cursor="cross")
        self.processed_canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # Comment Frame
        self.comment_frame = tk.Frame(root, height=100, bg="white")
        self.comment_frame.pack(side=tk.LEFT, fill=tk.BOTH, padx=60, pady=10)

        self.comment_scrollbar = Scrollbar(self.comment_frame)
        self.comment_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.comment_text = tk.Text(self.comment_frame, wrap=tk.WORD, yscrollcommand=self.comment_scrollbar.set, height=5)
        self.comment_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.comment_scrollbar.config(command=self.comment_text.yview)

        self.processed_canvas.bind("<Button-1>", self.canvas_click)
        self.processed_canvas.bind("<Double-1>", self.canvas_double_click)
        self.processed_canvas.bind("<ButtonRelease-1>", self.canvas_release)
        self.processed_canvas.bind("<B1-Motion>", self.move_bounding_box)
        self.root.bind("<Delete>", self.delete_bbox_keypress)
        self.original_canvas.bind("<ButtonPress-1>", self.pan_start)
        self.original_canvas.bind("<B1-Motion>", self.pan_move)
        self.detections = []
        self.bbox_handles = {}
        self.bbox_labels = {}
        self.detection_comments = {}
        self.selected_bbox = None
        self.move_start = None
        self.resizing = False
        self.resize_handle = None

        self.original_image = None
        self.processed_image = None
        self.dicom_image = None
        self.image_scale = 1
        self.original_zoom_factor = 1
        self.processed_zoom_factor = 1

        self.history = []
        self.initial_state = None

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            if file_path.lower().endswith('.dcm'):
                self.dicom_image = pydicom.dcmread(file_path)
                pixel_array = self.dicom_image.pixel_array
                pixel_array = self.normalize_pixel_array(pixel_array)
                image = Image.fromarray(pixel_array).convert('L')
                self.details_text.insert(tk.END, str(self.dicom_image))
            else:
                image = Image.open(file_path)

            self.original_image = image
            self.processed_image = image.copy()
            self.display_image(self.original_canvas, self.original_image)
            self.display_image(self.processed_canvas, self.processed_image)
            self.save_initial_state()

    def normalize_pixel_array(self, pixel_array):
        if np.issubdtype(pixel_array.dtype, np.integer):
            max_val = np.max(pixel_array)
            if max_val > 0:
                pixel_array = (pixel_array / max_val) * 255.0
            pixel_array = pixel_array.astype(np.uint8)
        elif np.issubdtype(pixel_array.dtype, np.float):
            pixel_array = (pixel_array - np.min(pixel_array)) / (np.max(pixel_array) - np.min(pixel_array))
            pixel_array = (pixel_array * 255).astype(np.uint8)
        return pixel_array

    def apply_filter(self):
        if not self.original_image:
            messagebox.showerror("Error", "Load an image first.")
            return

        image_np = np.array(self.original_image.convert("RGB"))
        results = self.model(image_np)

        self.detections.clear()
        for i, result in enumerate(results):
            for j, bbox in enumerate(result.boxes):
                x1, y1, x2, y2 = bbox.xyxy[0].int().tolist()
                class_name = result.names[bbox.cls[0].item()]
                confidence = bbox.conf[0].item()
                detection_id = len(self.detections)
                self.detections.append({
                    'id': detection_id,
                    'bbox': (x1, y1, x2, y2),
                    'class': class_name,
                    'confidence': confidence,
                    'comment': ''
                })

        self.processed_image = self.original_image.copy()
        self.draw_bounding_boxes()
        self.display_image(self.processed_canvas, self.processed_image)
        self.save_initial_state()

    def pan_start(self, event):
        self.original_canvas.scan_mark(event.x, event.y)

    def pan_move(self, event):
        self.original_canvas.scan_dragto(event.x, event.y, gain=1)
        self.sync_canvases()

    def sync_canvases(self):
        # Get the scroll region of the original canvas
        x0, x1 = self.original_canvas.xview()
        y0, y1 = self.original_canvas.yview()
        
        # Set the processed canvas view to match the original canvas view
        self.processed_canvas.xview_moveto(x0)
        self.processed_canvas.yview_moveto(y0)

    def canvas_click(self, event):
        x = self.processed_canvas.canvasx(event.x)
        y = self.processed_canvas.canvasy(event.y)
        clicked_bbox_id = None
        handle = None

        for bbox_id, bbox in enumerate(self.detections):
            x1, y1, x2, y2 = bbox['bbox']
            if x1 <= x <= x2 and y1 <= y <= y2:
                clicked_bbox_id = bbox_id
                handle = self.get_resize_handle(x, y, x1, y1, x2, y2)
                break

        if clicked_bbox_id is not None:
            self.selected_bbox = clicked_bbox_id
            self.move_start = (x, y)
            if handle:
                self.resizing = True
                self.resize_handle = handle
        else:
            self.selected_bbox = None

    def get_resize_handle(self, x, y, x1, y1, x2, y2):
        if abs(x - x1) < 10 and abs(y - y1) < 10:
            self.processed_canvas.config(cursor="sizing")
            return 'top_left'
        elif abs(x - x2) < 10 and abs(y - y1) < 10:
            self.processed_canvas.config(cursor="sizing")
            return 'top_right'
        elif abs(x - x1) < 10 and abs(y - y2) < 10:
            self.processed_canvas.config(cursor="sizing")
            return 'bottom_left'
        elif abs(x - x2) < 10 and abs(y - y2) < 10:
            self.processed_canvas.config(cursor="sizing")
            return 'bottom_right'
        return None

    def move_bounding_box(self, event):
        if self.selected_bbox is not None and self.move_start:
            x = self.processed_canvas.canvasx(event.x)
            y = self.processed_canvas.canvasy(event.y)
            dx = x - self.move_start[0]
            dy = y - self.move_start[1]

            bbox = self.detections[self.selected_bbox]['bbox']
            x1, y1, x2, y2 = bbox

            if self.resizing:
                if self.resize_handle == 'top_left':
                    x1 += dx
                    y1 += dy
                elif self.resize_handle == 'top_right':
                    x2 += dx
                    y1 += dy
                elif self.resize_handle == 'bottom_left':
                    x1 += dx
                    y2 += dy
                elif self.resize_handle == 'bottom_right':
                    x2 += dx
                    y2 += dy
            else:
                x1 += dx
                y1 += dy
                x2 += dx
                y2 += dy

            self.detections[self.selected_bbox]['bbox'] = (x1, y1, x2, y2)
            self.move_start = (x, y)

            self.processed_image = self.original_image.copy()
            self.draw_bounding_boxes()
            self.display_image(self.processed_canvas, self.processed_image)

    def canvas_release(self, event):
        self.selected_bbox = None
        self.move_start = None
        self.resizing = False
        self.resize_handle = None

    def canvas_double_click(self, event):
        x = self.processed_canvas.canvasx(event.x)
        y = self.processed_canvas.canvasy(event.y)
        clicked_bbox_id = None

        for bbox_id, bbox in enumerate(self.detections):
            x1, y1, x2, y2 = bbox['bbox']
            if x1 <= x <= x2 and y1 <= y <= y2:
                clicked_bbox_id = bbox_id
                break

        if clicked_bbox_id is not None:
            comment = simpledialog.askstring("Add Comment", "Enter your comment:")
            if comment:
                self.detections[clicked_bbox_id]['comment'] = comment
                self.processed_image = self.original_image.copy()
                self.draw_bounding_boxes()
                self.display_image(self.processed_canvas, self.processed_image)

    def delete_bbox_keypress(self, event):
        if self.selected_bbox is not None:
            del self.detections[self.selected_bbox]
            self.selected_bbox = None
            self.processed_image = self.original_image.copy()
            self.draw_bounding_boxes()
            self.display_image(self.processed_canvas, self.processed_image)

    def draw_bounding_boxes(self):
        draw = ImageDraw.Draw(self.processed_image)
        font = ImageFont.truetype("arial.ttf", 20)
        for detection in self.detections:
            x1, y1, x2, y2 = detection['bbox']
            label = f"{detection['class']} ({detection['confidence']:.2f})"
            draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
            draw.text((x1, y1), label, fill="yellow", font=font)
            if detection['comment']:
                draw.text((x1, y2), detection['comment'], fill="blue", font=font)

    def display_image(self, canvas, image):
        canvas.image = ImageTk.PhotoImage(image)
        canvas.create_image(0, 0, anchor=tk.NW, image=canvas.image)
        canvas.config(scrollregion=canvas.bbox(tk.ALL))

    def zoom_images(self, value):
        zoom_factor = float(value)
        self.processed_zoom_factor = zoom_factor
        if self.original_image:
            self.original_image = self.original_image.resize(
                (int(self.original_image.width * zoom_factor), int(self.original_image.height * zoom_factor)), Image.ANTIALIAS)
            self.processed_image = self.processed_image.resize(
                (int(self.processed_image.width * zoom_factor), int(self.processed_image.height * zoom_factor)), Image.ANTIALIAS)
            self.display_image(self.original_canvas, self.original_image)
            self.display_image(self.processed_canvas, self.processed_image)

    def undo(self):
        if self.history:
            state = self.history.pop()
            self.restore_state(state)

    def restore(self):
        if self.initial_state:
            self.restore_state(self.initial_state)

    def save_initial_state(self):
        self.initial_state = self.capture_state()

    def save_project(self):
        project = {
            'original_image': self.original_image,
            'processed_image': self.processed_image,
            'detections': self.detections,
            'comments': self.detection_comments
        }
        file_path = filedialog.asksaveasfilename(defaultextension=".pkl")
        if file_path:
            with open(file_path, 'wb') as file:
                pickle.dump(project, file)

    def load_project(self):
        file_path = filedialog.askopenfilename(filetypes=[("Pickle files", "*.pkl")])
        if file_path:
            with open(file_path, 'rb') as file:
                project = pickle.load(file)
            self.original_image = project['original_image']
            self.processed_image = project['processed_image']
            self.detections = project['detections']
            self.detection_comments = project['comments']
            self.display_image(self.original_canvas, self.original_image)
            self.display_image(self.processed_canvas, self.processed_image)
            self.draw_bounding_boxes()

    def export_as_pdf(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".pdf")
        if file_path:
            pdf = FPDF()
            pdf.add_page()
            temp_image_path = "temp_image.jpg"
            self.processed_image.save(temp_image_path)
            pdf.image(temp_image_path, x=10, y=10, w=pdf.w - 20)
            os.remove(temp_image_path)
            pdf.output(file_path)

    def export_modified_dicom(self):
        if self.dicom_image:
            file_path = filedialog.asksaveasfilename(defaultextension=".dcm")
            if file_path:
                self.dicom_image.PixelData = np.array(self.processed_image).tobytes()
                self.dicom_image.save_as(file_path)

    def toggle_details_visibility(self):
        if self.details_frame_visible:
            self.details_frame.pack_forget()
            self.details_toggle_button.config(text="⮞")
        else:
            self.details_frame.pack(side=tk.LEFT, fill=tk.Y)
            self.details_toggle_button.config(text="⮜")
        self.details_frame_visible = not self.details_frame_visible

    def capture_state(self):
        return {
            'original_image': self.original_image.copy(),
            'processed_image': self.processed_image.copy(),
            'detections': self.detections[:],
            'comments': self.detection_comments.copy()
        }

    def restore_state(self, state):
        self.original_image = state['original_image']
        self.processed_image = state['processed_image']
        self.detections = state['detections']
        self.detection_comments = state['comments']
        self.display_image(self.original_canvas, self.original_image)
        self.display_image(self.processed_canvas, self.processed_image)
        self.draw_bounding_boxes()

if __name__ == "__main__":
    root = tk.Tk()
    app = MedImagePro(root)
    root.mainloop()



0: 384x640 7 persons, 1 sports ball, 2114.5ms
Speed: 5.7ms preprocess, 2114.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 2112.5ms
Speed: 4.1ms preprocess, 2112.5ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Shawon\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Shawon\AppData\Local\Temp\ipykernel_16264\3358033244.py", line 270, in move_bounding_box
    self.draw_bounding_boxes()
  File "C:\Users\Shawon\AppData\Local\Temp\ipykernel_16264\3358033244.py", line 312, in draw_bounding_boxes
    draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
  File "c:\Users\Shawon\anaconda3\Lib\site-packages\PIL\ImageDraw.py", line 305, in rectangle
    self.draw.draw_rectangle(xy, ink, 0, width)
ValueError: x1 must be greater than or equal to x0
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Shawon\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Shawon\AppData\Local\Temp\ipykernel_16264\3358033244.py", line 270, in move_